In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'yeong_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
You must check this make_timeseries : time 

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.3787 - disc_loss: 0.3486 - rmse: 0.4705 - val_loss: 0.4596
Epoch 48/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.6134 - disc_loss: 0.3334 - rmse: 0.4339 - val_loss: 0.4726
Epoch 49/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.6260 - disc_loss: 0.3489 - rmse: 0.3629 - val_loss: 0.4371
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2229 - disc_loss: 0.3343 - rmse: 0.4100 - val_loss: 0.4018
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2263 - disc_loss: 0.3090 - rmse: 0.4144 - val_loss: 0.4104
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1355 - disc_loss: 0.3269 - rmse: 0.4027 - val_loss: 0.4341
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2836 - disc_loss: 0.3341 - rmse: 0.3625 - val_loss: 0.4497
Epoch 54/2000
1/1 [=================

Epoch 106/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0629 - disc_loss: 0.2488 - rmse: 0.3769 - val_loss: 0.3869
Epoch 107/2000
1/1 [==============================] - 0s 24ms/step - gen_loss: 7.5928 - disc_loss: 0.2832 - rmse: 0.3573 - val_loss: 0.3828
Epoch 108/2000
1/1 [==============================] - 0s 23ms/step - gen_loss: 8.3745 - disc_loss: 0.2791 - rmse: 0.3754 - val_loss: 0.3806
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8112 - disc_loss: 0.2611 - rmse: 0.3638 - val_loss: 0.3488
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8283 - disc_loss: 0.2500 - rmse: 0.3488 - val_loss: 0.3552
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3928 - disc_loss: 0.2742 - rmse: 0.3652 - val_loss: 0.3823
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8006 - disc_loss: 0.2488 - rmse: 0.3178 - val_loss: 0.3537
Epoch 113/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8398 - disc_loss: 0.2495 - rmse: 0.3236 - val_loss: 0.3474
Epoch 165/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6527 - disc_loss: 0.2469 - rmse: 0.3237 - val_loss: 0.3169
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8186 - disc_loss: 0.2359 - rmse: 0.3021 - val_loss: 0.3415
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1738 - disc_loss: 0.2491 - rmse: 0.2987 - val_loss: 0.3221
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6390 - disc_loss: 0.2470 - rmse: 0.3290 - val_loss: 0.3218
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3823 - disc_loss: 0.2312 - rmse: 0.3689 - val_loss: 0.3012
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8086 - disc_loss: 0.2432 - rmse: 0.3163 - val_loss: 0.3576
Epoch 171/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9011 - disc_loss: 0.2306 - rmse: 0.3058 - val_loss: 0.3723
Epoch 223/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4503 - disc_loss: 0.2385 - rmse: 0.3433 - val_loss: 0.3331
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8140 - disc_loss: 0.2450 - rmse: 0.2881 - val_loss: 0.3077
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5272 - disc_loss: 0.2480 - rmse: 0.3366 - val_loss: 0.3523
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5267 - disc_loss: 0.2174 - rmse: 0.3482 - val_loss: 0.3148
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2856 - disc_loss: 0.2108 - rmse: 0.3347 - val_loss: 0.3720
Epoch 228/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.0708 - disc_loss: 0.2236 - rmse: 0.3365 - val_loss: 0.3198
Epoch 229/2000
1/1 [===============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5445 - disc_loss: 0.1574 - rmse: 0.2687 - val_loss: 0.2904
Epoch 1615/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2269 - disc_loss: 0.1597 - rmse: 0.2642 - val_loss: 0.2858
Epoch 1616/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6380 - disc_loss: 0.1700 - rmse: 0.2937 - val_loss: 0.3065
Epoch 1617/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3261 - disc_loss: 0.1570 - rmse: 0.2483 - val_loss: 0.2790
Epoch 1618/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2369 - disc_loss: 0.1680 - rmse: 0.2577 - val_loss: 0.2652
Epoch 1619/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9026 - disc_loss: 0.1555 - rmse: 0.2281 - val_loss: 0.2780
Epoch 1620/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5094 - disc_loss: 0.1627 - rmse: 0.2853 - val_loss: 0.2485
Epoch 1621/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8910 - disc_loss: 0.1558 - rmse: 0.2449 - val_loss: 0.2542
Epoch 1673/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1121 - disc_loss: 0.1616 - rmse: 0.2524 - val_loss: 0.2262
Epoch 1674/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1297 - disc_loss: 0.1618 - rmse: 0.2648 - val_loss: 0.2762
Epoch 1675/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.4998 - disc_loss: 0.1576 - rmse: 0.2217 - val_loss: 0.2551
Epoch 1676/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5278 - disc_loss: 0.1596 - rmse: 0.2787 - val_loss: 0.2685
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7429 - disc_loss: 0.1516 - rmse: 0.2284 - val_loss: 0.2965
Epoch 1678/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9506 - disc_loss: 0.1623 - rmse: 0.2427 - val_loss: 0.2834
Epoch 1679/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.3908 - disc_loss: 0.1593 - rmse: 0.2697 - val_loss: 0.2389
Epoch 1731/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1530 - disc_loss: 0.1543 - rmse: 0.2323 - val_loss: 0.2296
Epoch 1732/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7113 - disc_loss: 0.1495 - rmse: 0.2214 - val_loss: 0.2954
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9772 - disc_loss: 0.1524 - rmse: 0.2448 - val_loss: 0.2477
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5215 - disc_loss: 0.1627 - rmse: 0.2501 - val_loss: 0.2619
Epoch 1735/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0547 - disc_loss: 0.1627 - rmse: 0.2536 - val_loss: 0.2540
Epoch 1736/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0538 - disc_loss: 0.1558 - rmse: 0.2359 - val_loss: 0.2366
Epoch 1737/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5467 - disc_loss: 0.1534 - rmse: 0.2494 - val_loss: 0.3018
Epoch 1789/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3648 - disc_loss: 0.1566 - rmse: 0.2495 - val_loss: 0.3015
Epoch 1790/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3801 - disc_loss: 0.1559 - rmse: 0.2710 - val_loss: 0.2754
Epoch 1791/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 3.5648 - disc_loss: 0.1686 - rmse: 0.2521 - val_loss: 0.2780
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3712 - disc_loss: 0.1548 - rmse: 0.2304 - val_loss: 0.2935
Epoch 1793/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3329 - disc_loss: 0.1554 - rmse: 0.2393 - val_loss: 0.2763
Epoch 1794/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6335 - disc_loss: 0.1523 - rmse: 0.3103 - val_loss: 0.2925
Epoch 1795/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7019 - disc_loss: 0.1533 - rmse: 0.2700 - val_loss: 0.2449
Epoch 1847/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4394 - disc_loss: 0.1702 - rmse: 0.2500 - val_loss: 0.2487
Epoch 1848/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2923 - disc_loss: 0.1627 - rmse: 0.2512 - val_loss: 0.2396
Epoch 1849/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4977 - disc_loss: 0.1568 - rmse: 0.2558 - val_loss: 0.2496
Epoch 1850/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1635 - disc_loss: 0.1472 - rmse: 0.2421 - val_loss: 0.2982
Epoch 1851/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1714 - disc_loss: 0.1540 - rmse: 0.2615 - val_loss: 0.2492
Epoch 1852/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2863 - disc_loss: 0.1561 - rmse: 0.2398 - val_loss: 0.2459
Epoch 1853/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [16]:
print(real_df_all.shape)
rnn_target_column = 'chl-a'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(52608, 48)
chl-a


In [17]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 48) val_df.shape :  (10522, 48) test_df.shape: (5261, 48)


In [18]:
#fake_df.shape[0]/8760

In [19]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [22]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  7
out_num_features :  1


In [23]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [24]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/chlorophyll-a/


In [25]:
rnn_epochs

20

## 모델 학습

In [26]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 1.7465 - mean_absolute_error: 0.8740 - nse: 0.0816
Epoch 00001: val_nse improved from -inf to 0.40837, saving model to save/best_model.h5
10/10 [==============================] - 9s 885ms/step - loss: 1.7465 - mean_absolute_error: 0.8740 - nse: 0.0816 - val_loss: 0.5776 - val_mean_absolute_error: 0.5657 - val_nse: 0.4084
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.8904 - mean_absolute_error: 0.6430 - nse: 0.5616
Epoch 00002: val_nse improved from 0.40837 to 0.44829, saving model to save/best_model.h5
10/10 [==============================] - 9s 873ms/step - loss: 0.8904 - mean_absolute_error: 0.6430 - nse: 0.5616 - val_loss: 0.5326 - val_mean_absolute_error: 0.5297 - val_nse: 0.4483
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.6541 - mean_absolute_error: 0.5402 - nse: 0.6782
Epoch 00003: val_nse improved from 0.44829 to 0.49712, saving model to save/best_model.h5
10/10 

10/10 [==============================] - ETA: 0s - loss: 0.3871 - mean_absolute_error: 0.4272 - nse: 0.7911
Epoch 00022: val_nse did not improve from 0.61253
10/10 [==============================] - 8s 849ms/step - loss: 0.3871 - mean_absolute_error: 0.4272 - nse: 0.7911 - val_loss: 0.5876 - val_mean_absolute_error: 0.5002 - val_nse: 0.3933
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.3711 - mean_absolute_error: 0.4236 - nse: 0.8143
Epoch 00023: val_nse did not improve from 0.61253
10/10 [==============================] - 9s 869ms/step - loss: 0.3711 - mean_absolute_error: 0.4236 - nse: 0.8143 - val_loss: 0.4892 - val_mean_absolute_error: 0.4668 - val_nse: 0.4943
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.4015 - mean_absolute_error: 0.4355 - nse: 0.7702
Epoch 00024: val_nse did not improve from 0.61253
10/10 [==============================] - 9s 868ms/step - loss: 0.4015 - mean_absolute_error: 0.4355 - nse: 0.7702 - val_loss: 0.5545

10/10 [==============================] - ETA: 0s - loss: 0.2836 - mean_absolute_error: 0.3639 - nse: 0.8402
Epoch 00045: val_nse did not improve from 0.61253
10/10 [==============================] - 9s 876ms/step - loss: 0.2836 - mean_absolute_error: 0.3639 - nse: 0.8402 - val_loss: 0.8034 - val_mean_absolute_error: 0.5943 - val_nse: 0.1774
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.3000 - mean_absolute_error: 0.3740 - nse: 0.8368
Epoch 00046: val_nse did not improve from 0.61253
10/10 [==============================] - 9s 860ms/step - loss: 0.3000 - mean_absolute_error: 0.3740 - nse: 0.8368 - val_loss: 0.6906 - val_mean_absolute_error: 0.5616 - val_nse: 0.2891
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.2939 - mean_absolute_error: 0.3675 - nse: 0.8383
Epoch 00047: val_nse did not improve from 0.61253
10/10 [==============================] - 9s 871ms/step - loss: 0.2939 - mean_absolute_error: 0.3675 - nse: 0.8383 - val_loss: 0.6487

## core / window.py / 

In [30]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  50.385384
predict_day :  5
------------------------
13.613805 395.42404 19.885271
nse3 :  [0.56855094]
pbias3 :  [-3.9887085]
save/yeong/models/chlorophyll-a/
year : 2015 ~ 2020
run :  range(0, 1)
target :  chl-a
length :  52608
train=7, test=1, val=2
---------------
[0.56855094]
[3.9887085]


In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
